## 机器翻译与数据集
语⾔模型是⾃然语⾔处理的关键，⽽机器翻译是语⾔模型最成功的基准测试。因为机器翻译正是将输⼊序列
转换成输出序列的 序列转换模型（sequence transduction）的核⼼问题。序列转换模型在各类现代⼈⼯智能
应⽤中发挥着⾄关重要的作⽤，因此我们将其做为本章剩余部分和 10节的重点。为此，本节将介绍机器翻译
问题及其后⽂需要使⽤的数据集。

机器翻译（machine translation）指的是将序列从⼀种语⾔⾃动翻译成另⼀种语⾔。事实上，这个研究领域可
以追溯到数字计算机发明后不久的20世纪40年代，特别是在第⼆次世界⼤战中使⽤计算机破解语⾔编码。⼏
⼗年来，在使⽤神经⽹络进⾏端到端学习的兴起之前，统计学⽅法在这⼀领域⼀直占据主导地位 (Brown et
al., 1990, Brown et al., 1988)。因为统计机器翻译（statistical machine translation）涉及了翻译模型和语⾔模
型等组成部分的统计分析，因此基于神经⽹络的⽅法通常被称为神经机器翻译（neuralmachinetranslation），
⽤于将两种翻译模型区分开来。

本书的关注点是神经⽹络机器翻译⽅法，强调的是端到端的学习。与 8.3节中的语料库是单⼀语⾔的语⾔模
型问题存在不同，机器翻译的数据集是由源语⾔和⽬标语⾔的⽂本序列对组成的。因此，我们需要⼀种完全
不同的⽅法来预处理机器翻译数据集，⽽不是复⽤语⾔模型的预处理程序。下⾯，我们看⼀下如何将预处理
后的数据加载到⼩批量中⽤于训练。


In [ ]:
import os
import torch
from d2l import torch as d2l

### 下载和预处理数据集
⾸先，下载⼀个由Tatoeba项⽬的双语句⼦对 113 组成的“英－法”数据集，数据集中的每⼀⾏都是制表符分隔
的⽂本序列对，序列对由英⽂⽂本序列和翻译后的法语⽂本序列组成。请注意，每个⽂本序列可以是⼀个句
⼦，也可以是包含多个句⼦的⼀个段落。在这个将英语翻译成法语的机器翻译问题中，英语是源语⾔（source
language），法语是⽬标语⾔（target language）。

In [ ]:
# @save
d2l.DATA_HUB['fra-eng'] = (d2l.DATA_URL + 'fra-eng.zip',
                           '94646ad1522d915e7b0f9296181140edcf86a4f5')

# d2l.read_data_nmt()
def read_data_nmt():
    """载入“英语-法语”数据集"""
    data_dir = d2l.download_extract('fra-eng')
    with open(os.path.join(data_dir, 'fra.txt'), 'r',
              encoding='utf-8') as f:
        return f.read()
    
raw_text = read_data_nmt()
print(raw_text[:75])

下载数据集后，原始⽂本数据需要经过⼏个预处理步骤。例如，我们⽤空格代替不间断空格（non-breaking
space），使⽤⼩写字⺟替换⼤写字⺟，并在单词和标点符号之间插⼊空格。

In [ ]:
# d2l.preprocess_nmt(text)
def preprocess_nmt(text):
    """预处理“英语-法语”数据集"""
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '
    
    # 使用空格替换不间断空格
    # 使用小写字母替换大写字母
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    # 在单词和标点符号之间插入空格
    out = [' ' + char if i > 0 and no_space(char, text[i-1]) else char
           for i, char in enumerate(text)]
    
    return ''.join(out)

text = preprocess_nmt(raw_text)
print(text[:80])

### 词元化
与 8.3节中的字符级词元化不同，在机器翻译中，我们更喜欢单词级词元化（最先进的模型可能使⽤更⾼级
的词元化技术）。下⾯的tokenize_nmt函数对前num_examples个⽂本序列对进⾏词元，其中每个词元要么是
⼀个词，要么是⼀个标点符号。此函数返回两个词元列表：source和target：source[i]是源语⾔（这⾥是英
语）第i个⽂本序列的词元列表，target[i]是⽬标语⾔（这⾥是法语）第i个⽂本序列的词元列表。

In [ ]:
# d2l.tokenize_nmt(text, num_examples=None)
def tokenize_nmt(text, num_examples=None):
    """词元化“英语-法语”数据数据集"""
    source, target = [], []
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
        if len(parts) == 2:
            source.append(parts[0].split(' '))
            target.append(parts[1].split(' '))
    return source, target

In [ ]:
source, target = tokenize_nmt(text)
source[:6], target[:6]

让我们绘制每个⽂本序列所包含的词元数量的直⽅图。在这个简单的“英－法”数据集中，⼤多数⽂本序列
的词元数量少于20个。

In [ ]:
# d2l.show_list_len_pair_hist()
def show_list_len_pair_hist(legend, xlabel, ylabel, xlist, ylist):
    """绘制列表长度对的直方图"""
    d2l.set_figsize()
    _, _, patches = d2l.plt.hist(
        [[len(l) for l in xlist], [len(l) for l in ylist]])
    d2l.plt.xlabel(xlabel)
    d2l.plt.ylabel(ylabel)
    for patch in patches[1].patches:
        patch.set_hatch('/')
    d2l.plt.legend(legend)


In [ ]:
show_list_len_pair_hist(['source', 'target'], '# tokens per sequence',
                        'count', source, target)

### 词表
由于机器翻译数据集由语⾔对组成，因此我们可以分别为源语⾔和⽬标语⾔构建两个词表。使⽤单词级词
元化时，词表⼤⼩将明显⼤于使⽤字符级词元化时的词表⼤⼩。为了缓解这⼀问题，这⾥我们将出现次数少
于2次的低频率词元视为相同的未知（“(unk)”）词元。除此之外，我们还指定了额外的特定词元，例如在⼩
批量时⽤于将序列填充到相同⻓度的填充词元（“(pad)”），以及序列的开始词元（“(bos)”）和结束词元
（“(eos)”）。这些特殊词元在⾃然语⾔处理任务中⽐较常⽤。

In [ ]:
src_vocab = d2l.Vocab(source, min_freq=2,
                      reserved_tokens=['<pad>', '<bos>', '<eos>'])
len(src_vocab)

### 加载数据集
回想⼀下，语⾔模型中的序列样本都有⼀个固定的⻓度，⽆论这个样本是⼀个句⼦的⼀部分还是跨越了多个
句⼦的⼀个⽚断。这个固定⻓度是由 8.3节中的 num_steps（时间步数或词元数量）参数指定的。在机器翻译
中，每个样本都是由源和⽬标组成的⽂本序列对，其中的每个⽂本序列可能具有不同的⻓度。

为了提⾼计算效率，我们仍然可以通过截断（truncation）和 填充（padding）⽅式实现⼀次只处理⼀个⼩批
量的⽂本序列。假设同⼀个⼩批量中的每个序列都应该具有相同的⻓度num_steps，那么如果⽂本序列的词
元数⽬少于num_steps时，我们将继续在其末尾添加特定的“(pad)”词元，直到其⻓度达到num_steps；反
之，我们将截断⽂本序列时，只取其前num_steps 个词元，并且丢弃剩余的词元。这样，每个⽂本序列将具
有相同的⻓度，以便以相同形状的⼩批量进⾏加载。
    
如前所述，下⾯的truncate_pad函数将截断或填充⽂本序列。

In [ ]:
# d2l.truncate_pad(line, num_steps, padding_token)
def truncate_pad(line, num_steps, padding_token):
    """截断或填充文本序列"""
    if len(line) > num_steps:
        return line[:num_steps] # 截断
    return line + [padding_token] * (num_steps - len(line)) # 填充

In [ ]:
truncate_pad(src_vocab[source[0]], 10, src_vocab['<pad>'])

现在我们定义⼀个函数，可以将⽂本序列转换成⼩批量数据集⽤于训练。我们将特定的“(eos)”词元添加
到所有序列的末尾，⽤于表⽰序列的结束。当模型通过⼀个词元接⼀个词元地⽣成序列进⾏预测时，⽣成的
“(eos)”词元说明完成了序列输出⼯作。此外，我们还记录了每个⽂本序列的⻓度，统计⻓度时排除了填充
词元，在稍后将要介绍的⼀些模型会需要这个⻓度信息。

In [ ]:
# d2l.build_array_nmt(lines, vocab, num_steps)
def build_array_nmt(lines, vocab, num_steps):
    """将机器翻译的文本序列转换成小批量"""
    lines = [vocab[l] for l in lines]
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([truncate_pad(
        l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).type(torch.int32).sum(1)
    return array, valid_len

### 训练模型
最后，我们定义load_data_nmt函数来返回数据迭代器，以及源语⾔和⽬标语⾔的两种词表。

In [ ]:
# d2l.load_data_nmt(batch_size, num_steps, num_examples=600)
def load_data_nmt(batch_size, num_steps, num_examples=600):
    """返回翻译数据集的迭代器和词表"""
    text = preprocess_nmt(read_data_nmt())
    source, target = tokenize_nmt(text, num_examples)
    src_vocab = d2l.Vocab(source, min_freq=2,
                          reserved_tokens=['<pad>', '<bos>', '<eos>'])
    tgt_vocab = d2l.Vocab(target, min_freq=2,
                          reserved_tokens=['<pad>', '<bos>', '<eos>'])
    src_array, src_valid_len = build_array_nmt(source, src_vocab, num_steps)
    tgt_array, tgt_valid_len = build_array_nmt(target, tgt_vocab, num_steps)
    
    data_arrays = (src_array, src_valid_len, tgt_array, tgt_valid_len)
    data_iter = d2l.load_array(data_arrays, batch_size)
    return data_iter, src_vocab, tgt_vocab

In [ ]:
# 下⾯我们读出“英语－法语”数据集中的第⼀个⼩批量数据。
train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size=2, num_steps=8)
for X, X_valid_len, Y, Y_valid_len in train_iter:
    print('X:', X.type(torch.int32))
    print('X的有效长度:', X_valid_len)
    print('Y:', Y.type(torch.int32))
    print('Y的有效长度:', Y_valid_len)
    break

⼩结

    • 机器翻译指的是将⽂本序列从⼀种语⾔⾃动翻译成另⼀种语⾔。
    • 使⽤单词级词元化时的词表⼤⼩，将明显⼤于使⽤字符级词元化时的词表⼤⼩。为了缓解这⼀问题，我们可以将低频词元视为相同的未知词元。
    • 通过截断和填充⽂本序列，可以保证所有的⽂本序列都具有相同的⻓度，以便以⼩批量的⽅式加载。